### Chatbots with History
ChatGPT took the world by storm by exposing a powerful language model with a new interface - chat. There are several components that go into building a chatbot.

- The model - you can construct a chatbot from a normal language model or a Chat Model. The important thing to remember is that even if you are using a Chat Model, the API itself is stateless, meaning it won't remember previous interactions - you have to pass them in.
- PromptTemplate - this will guide how your chatbot acts. Are they sassy? Helpful? These can be used to give your chatbot some character.
- Memory - as mentioned above, the models themselves are stateless. Memory brings some concept of state to the table, allowing it remember previous interactions

Chatbots are often very powerful and more differentiated when combined with other sources of data. The same techniques that underpin "Question Answering Over Docs" can also be used here to give your chatbot access to that data.

#### Set Environment Variables

In [1]:
import os  
import json  
import openai
from Utilities.envVars import *

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = f"https://{OpenAiService}.openai.azure.com"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint
davincimodel = OpenAiDavinci


#### Let's chat on same index document we created in previous notebook

In [21]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms.openai import AzureOpenAI, OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from Utilities.cogSearch import performCogSearch
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from Utilities.cogSearchRetriever import CognitiveSearchRetriever
from langchain.chains import LLMChain

embeddingModelType = "openai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
    openai.api_type = "azure"
    openai.api_key = OpenAiKey
    openai.api_version = OpenAiVersion
    openai.api_base = f"https://{OpenAiService}.openai.azure.com"

    llm = AzureOpenAI(deployment_name=OpenAiDavinci,
            temperature=temperature,
            openai_api_key=OpenAiKey,
            max_tokens=tokenLength,
            batch_size=10, 
            max_retries=12)

    logging.info("LLM Setup done")
    embeddings = OpenAIEmbeddings(model=OpenAiEmbedding, chunk_size=1, openai_api_key=OpenAiKey)
elif embeddingModelType == "openai":
    openai.api_type = "open_ai"
    openai.api_base = "https://api.openai.com/v1"
    openai.api_version = '2020-11-07' 
    openai.api_key = OpenAiApiKey
    llm = OpenAI(temperature=temperature,
            openai_api_key=OpenAiApiKey,
            max_tokens=tokenLength)
    embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)


In [7]:
#We can now create a memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [10]:
topK = 3

# We can now create a retriever object, which is neccessary to retrieve documents from the index.
retriever = CognitiveSearchRetriever(content_key="content",
                                                  service_name=SearchService,
                                                  api_key=SearchKey,
                                                  index_name=indexName,
                                                  topK=topK)

## Use the ConversationalRetrievalChain to combine the LLM, retriever, and memory into a single object.
qa = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)
query = "What is Microsoft Fabric"
answer = qa({"question": query})
outputAnswer = answer['answer']
print(outputAnswer)

 Microsoft Fabric is a comprehensive set of analytics experiences designed to work together seamlessly. It includes industry-leading experiences in the categories of data engineering, data factory, and data science. It also includes features such as OneLake, workspaces, and a help pane.


In [11]:
query = "Does it support real-time analytics use-cases?"
answer = qa({"question": query})
outputAnswer = answer['answer']
print(outputAnswer)

 Yes, Microsoft Fabric supports real-time analytics use-cases. It provides Synapse Real-Time Analytics, which includes a KQL database and KQL Queryset to query data in the Data Explorer database.


In [14]:
# In the above example, we used a Memory object to track chat history. We can also just pass it in explicitly. 
# In order to do this, we need to initialize a chain without any memory object.
chat_history = []

qaNoMemory = ConversationalRetrievalChain.from_llm(llm, retriever)
query = "What is Microsoft Fabric"
answer = qaNoMemory({"question": query, "chat_history": chat_history})
outputAnswer = answer['answer']
print(outputAnswer)

 Microsoft Fabric is a comprehensive set of analytics experiences designed to work together seamlessly. It includes industry-leading experiences in the categories of data engineering, data factory, and data science. It also includes OneLake, a unified storage system for all developers, and a hierarchical organizational structure to simplify management across organizations.


In [15]:
chat_history = [(query, answer["answer"])]
query = "Does it support real-time analytics use-cases?"
answer = qaNoMemory({"question": query, "chat_history": chat_history})
outputAnswer = answer['answer']
print(outputAnswer)

 Yes, Microsoft Fabric supports real-time analytics use-cases. Synapse Real-Time Analytics includes a KQL database and KQL Queryset which allow users to query data and manipulate query results.


#### You can also try using other Chaintype

In [25]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [32]:
chainType = "map_reduce"
questionGenerator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
docChain = load_qa_chain(llm, chain_type=chainType)

mapRChain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=questionGenerator,
    combine_docs_chain=docChain,
)

In [33]:
chat_history = []

query = "What is Microsoft Fabric"
answer = mapRChain({"question": query, "chat_history": chat_history})
outputAnswer = answer['answer']
print(outputAnswer)

 Microsoft Fabric is a comprehensive set of analytics experiences designed to work together seamlessly, including Data Engineering, Data Factory, and Data Science. It includes OneLake, which is hierarchical in nature to simplify management across an organization, and provides context sensitive help in the right rail of a browser.


In [35]:
chat_history = [(query, answer["answer"])]
query = "Does it support real-time analytics use-cases?"
answer = mapRChain({"question": query, "chat_history": chat_history})
outputAnswer = answer['answer']
print(outputAnswer)

 Yes, Microsoft Fabric supports real-time analytics use-cases.
